In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS


import sys 
sys.path.insert(1, "../")
from workloads.util import use_results, use_dataset

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'workloads'

In [6]:
dataset_dir = use_dataset("ml-100k")

NameError: name 'use_dataset' is not defined

In [2]:
ratings_path = "~/Downloads/ml-100k/u.data"
df = pd.read_csv(ratings_path, sep = '\t').sort_values('timestamp')

FileNotFoundError: [Errno 2] No such file or directory: '/home/eecs/wooders/Downloads/ml-100k/u.data'

In [218]:
df

,user_id,movie_id,rating,timestamp
214,259,255,4,874724710
83965,259,286,4,874724727
43027,259,298,4,874724754
21396,259,185,4,874724781
82655,259,173,4,874724843
...,...,...,...,...
46773,729,689,4,893286638
73008,729,313,3,893286638
46574,729,328,3,893286638
64312,729,748,4,893286638


In [219]:
len(df)

100000

In [220]:
squash_amount = 100

In [221]:
new_timestamp = [i // squash_amount for i in range(1, len(df)+1)]
new_timestamp

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [222]:
new_df = df.drop('timestamp', 1)
new_df['timestamp'] = new_timestamp

/var/folders/p5/zwk89rgd0cdf07_7k1wdqjgw0000gn/T/ipykernel_56430/405703479.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  new_df = df.drop('timestamp', 1)


In [223]:
new_df

,user_id,movie_id,rating,timestamp
214,259,255,4,0
83965,259,286,4,0
43027,259,298,4,0
21396,259,185,4,0
82655,259,173,4,0
...,...,...,...,...
46773,729,689,4,999
73008,729,313,3,999
46574,729,328,3,999
64312,729,748,4,999


In [224]:
def train_stream_test_split(df):
    columns = ["user_id", "movie_id", "rating", "timestamp"]
    als_data = []
    stream_data = []
    for user_id in df["user_id"].unique():
        user_df = df[df["user_id"] == user_id].sort_values("timestamp")
        count = 0
        for row in user_df.itertuples():
            data = [row.user_id, row.movie_id, row.rating, row.timestamp]
            if count < len(user_df) / 2:
                als_data.append(data)
            else:
                stream_data.append(data)
            count += 1
    als_df = pd.DataFrame(data=als_data, columns=columns).sort_values("timestamp")
    movie_list = set(als_df['movie_id'])
    stream_df = pd.DataFrame(data=stream_data, columns=columns).sort_values("timestamp")
    # Drop movies not seen before from stream
    stream_df = stream_df.drop(stream_df[stream_df['movie_id'].map(lambda x: x not in movie_list)].index)
    test_df = stream_df.copy()
    return als_df, stream_df, test_df

In [225]:
als_df, stream_df, test_df = train_stream_test_split(new_df)

In [226]:
spark = SparkSession.builder.master('local').appName('als').getOrCreate()

In [227]:
spark_als_df = spark.createDataFrame(als_df) 
spark_stream_df = spark.createDataFrame(stream_df)

In [228]:
als = ALS(
         userCol="user_id", 
         itemCol="movie_id",
         ratingCol="rating", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop",
         rank=150,
         maxIter=10,
         regParam=.1
)

In [229]:
model=als.fit(spark_als_df)

In [230]:
predictions = model.transform(spark_stream_df)
evaluator = RegressionEvaluator().setMetricName("rmse").setLabelCol("rating").setPredictionCol("prediction")
rmse = evaluator.evaluate(predictions)
rmse

0.9806914059604086

In [231]:
dir_path = "~/movielens/ralf/"

In [232]:
model.itemFactors.toPandas().sort_values('id').to_csv(dir_path + 'movie_features.csv', index = False)

/opt/homebrew/Caskroom/miniconda/base/envs/myenv/lib/python3.8/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [233]:
model.userFactors.toPandas().sort_values('id').to_csv(dir_path + 'user_features.csv', index = False)

In [234]:
stream_df = stream_df.sort_values('timestamp')
new_timestamp = [i // squash_amount for i in range(1, len(stream_df)+1)]
stream_df['timestamp'] = new_timestamp

In [235]:
stream_df

,user_id,movie_id,rating,timestamp
127,712,506,3,0
128,712,60,1,0
129,712,729,5,0
130,712,941,5,0
131,712,568,5,0
...,...,...,...,...
49755,729,288,2,494
49757,729,346,1,494
49754,729,879,3,494
49733,683,315,4,494


In [236]:
print(len(stream_df))

49442


In [237]:
stream_df.to_csv(dir_path + 'ratings.csv', index = False)